In [1]:
'''
Guide to use of program
Begin by running the below chunks, it should trigger instructions and entry boxes to follow

Follow Steps:

    1. Begin by entering the current date to consider most up-to-date data (Ensure it is in YYYY/MM/DD format)
    2. Begin entering the Stock Symbol (Ticker) that constitute your portfolio in ALL CAPS.
        - If you have mistakenly entered the wrong ticker, enter "Remove" to remove the last added ticker
        - Once all tickers have been entered, enter "Done" to proceed
    3. A loading bar should appear while the program is generating possible combinations of weights to create the efficient frontier
    4. Once it is done loading the overall return, variance and optimal weightage for your portfolio will appear.
'''

'\nGuide to use of program\nBegin by running the below chunks, it should trigger instructions and entry boxes to follow\n\nFollow Steps:\n\n    1. Begin by entering the current date to consider most up-to-date data (Ensure it is in YYYY/MM/DD format)\n    2. Begin entering the Stock Symbol (Ticker) that constitute your portfolio in ALL CAPS.\n        - If you have mistakenly entered the wrong ticker, enter "Remove" to remove the last added ticker\n        - Once all tickers have been entered, enter "Done" to proceed\n    3. A loading bar should appear while the program is generating possible combinations of weights to create the efficient frontier\n    4. Once it is done loading the overall return, variance and optimal weightage for your portfolio will appear.\n'

In [6]:
# Load Packages
import numpy as np
import pandas as pd
from pandas_datareader import data as Data
import matplotlib.pyplot as plt
from time import sleep
from tqdm import tqdm
import re
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from pypfopt import objective_functions
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
%matplotlib inline

def read_data():
    total_portfolio_value = input("Enter total portfolio value ($): ")
    while not total_portfolio_value.isnumeric():
        print("This does not appear to be a valid value, try again")
        total_portfolio_value = input("Enter total portfolio value ($): ")
    dateRegex = re.compile(r'(\d\d\d\d)/(\d\d)/(\d\d)')
    current_date = input("Enter current date (YYYY/MM/DD): ")
    while not dateRegex.findall(current_date):
        print("This does not appear to be a valid date, try again")
        current_date = input("Enter current date (YYYY/MM/DD): ")
    lookback = input("Enter lookback period (Years): ")
    while not lookback.isdigit():
        print("This does not appear to be a number of years, try again")
        lookback = input("Enter lookback period (Years): ")
    finished_entry = False
    stock_list = []
    start_date = str(int(current_date[0:4]) - int(lookback)) + current_date[4:]
    
    #print(start_date, current_date)
    print("Enter all stocks in your portfolio, once all have been entered, input 'Done'")
    while finished_entry == False:
        next_stock = input("Enter next stock: ")
        
        if next_stock == "Done":
            finished_entry = True
        elif next_stock == "Remove":
            a = stock_list.pop()
            print(a + " has been removed from list")
        else:
            try:
                Data.DataReader([next_stock], 'yahoo', start= start_date, end= current_date)
                stock_list.append(next_stock)
            except:
                print("This does not appear to be a real stock, try again")
    return current_date, start_date, stock_list, float(total_portfolio_value)



In [7]:
def main():
    current_date, start_date, stock_list, total_portfolio_value = read_data()
    df = Data.DataReader(stock_list, 'yahoo', start= start_date, end= current_date)
    df = df['Close']
    latest_prices = get_latest_prices(df)
    mu = mean_historical_return(df)
    S = CovarianceShrinkage(df).ledoit_wolf()
    ef = EfficientFrontier(mu, S)
    ef.add_objective(objective_functions.L2_reg, gamma=0.1)
    w = ef.max_sharpe()
    da = DiscreteAllocation(w, latest_prices, total_portfolio_value = total_portfolio_value)
    allocation, leftover = da.greedy_portfolio()
    print("For a total portfolio value of USD: " + str(total_portfolio_value))
    print("Your discrete allocation should be: ")
    for a,b in allocation.items():
        print(a + ": " + str(b))

if __name__ == "__main__":
    main()

Enter total portfolio value ($): 4700
Enter current date (YYYY/MM/DD): 2021/10/28
Enter lookback period (Years): 3
Enter all stocks in your portfolio, once all have been entered, input 'Done'
Enter next stock: FB
Enter next stock: GOOGL
Enter next stock: AAPL
Enter next stock: NKE
Enter next stock: Done
For a total portfolio value of USD: 4700.0
Your discrete allocation should be: 
AAPL: 9
NKE: 6
FB: 1


C:\Users\alvin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pypfopt\efficient_frontier\efficient_frontier.py:252: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(
